In [13]:
# importing required libraries
import numpy as np
import pandas as pd

import datetime as dt
import yfinance as yf
import pandas_datareader.data as web
import cpi
import wbdata

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from scipy.optimize import minimize

In [14]:
# customize seed for different stocks
np.random.seed(42)

# list of all potential stocks (can be customized)
ticker_list = ['AAPL','AXP', 'BAC', 'C', 'CSCO', 'GS', 'IBM', 'INTC', 'JPM', 'MSFT', 'NVDA', 'CRM', 'QCOM', 'NOW', 'ORCL', 'AVGO', 'TXN', 'GDDY',
               'WIX', 'TSM', 'TSLA', 'SNOW', 'HUBS', 'DOCU', 'MS']

# selecting 15 random stocks
stock_list = np.random.choice(ticker_list,15,replace=False)
print(f'These are the fifteen stocks assigned to you: {" ".join(stock_list)}')

# sorting the list from A-Z
stock_list = list(np.sort(stock_list))

These are the fifteen stocks assigned to you: JPM TXN AAPL DOCU CRM MSFT NOW AXP HUBS GS BAC QCOM AVGO C CSCO


In [15]:
# finding optimal historical portfolio since 2000 to today, using 15 stocks. 
# Assume no short-selling constraints

# defining start and end date
start = dt.datetime(2000, 1, 1)
end = dt.datetime(2024, 12, 31)

# getting returns of the stocks
# calculates returns using adjusted close price
# ensures that we have returns of only those dates where all companies have available information
returns = yf.download(stock_list, start-pd.offsets.BDay(1), end+pd.offsets.BDay(1), auto_adjust=False)['Adj Close'].pct_change().dropna()

# gets all returns Close, high, low, open, volume
#returns = yf.download(stock_list, start, end)

# getting monthly returns

# S&P 500
sp500 = yf.Ticker("^GSPC")
sp500_data = sp500.history(period="25y")
sp500_data['SP500'] = sp500_data['Close'].pct_change()
sp500_data = sp500_data.drop(columns=["Open", "High", "Low", "Close", "Volume", "Dividends", "Stock Splits"])


sp500_data.index = sp500_data.index.strftime('%m-%d-%Y')
sp500_data.index = pd.to_datetime(sp500_data.index)

# sp500_mon = sp500_data.resample(rule = 'ME').apply(lambda x: x.add(1).prod().sub(1))



[*********************100%***********************]  15 of 15 completed


In [16]:
returns = pd.merge(returns, sp500_data, left_index = True, right_index = True )

returns_mon = returns.resample(rule = 'ME').apply(lambda x: x.add(1).prod().sub(1))
returns_mon

,AAPL,AVGO,AXP,BAC,C,CRM,CSCO,DOCU,GS,HUBS,JPM,MSFT,NOW,QCOM,TXN,SP500
Date,,,,,,,,,,,,,,,,
2018-04-30,0.018113,-0.014434,-0.008634,-0.007629,-0.010436,0.006405,-0.009394,-0.027687,-0.006130,-0.018081,-0.005667,-0.024003,0.018140,-0.001957,-0.010536,-0.008187
2018-05-31,0.135124,0.098727,-0.004557,-0.025446,-0.018520,0.068931,-0.035674,0.289412,-0.048887,0.144476,-0.016272,0.061467,0.069038,0.151289,0.109965,0.021608
2018-06-30,-0.009418,-0.024502,-0.003052,-0.029270,0.003448,0.054666,0.007492,0.063040,-0.023508,0.034654,-0.026259,-0.002327,-0.028940,-0.034412,-0.014834,0.004842
2018-07-31,0.027983,-0.086012,0.019156,0.095424,0.074268,0.005499,-0.009537,0.017941,0.076438,-0.010367,0.109162,0.075753,0.020235,0.142017,0.015296,0.036022
2018-08-31,0.200422,-0.012355,0.064911,0.001619,-0.002795,0.113234,0.129582,0.158442,0.004918,0.157937,-0.003219,0.062993,0.115935,0.072086,0.009702,0.030263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,0.032353,0.013319,0.022170,0.010915,-0.025235,-0.022798,0.043137,0.067231,0.008327,0.004104,0.056391,-0.001095,0.049866,-0.031224,0.051666,0.022835
2024-09-30,0.017467,0.062937,0.048521,-0.019909,-0.000639,0.083985,0.053027,0.048640,-0.029672,0.065181,-0.062011,0.031548,0.046070,-0.024936,-0.036251,0.020197
2024-10-31,-0.030429,-0.015826,-0.001528,0.053931,0.025080,0.064521,0.036989,0.117410,0.045808,0.043623,0.058901,-0.055659,0.043158,-0.042811,-0.010033,-0.009897


In [17]:
# pd.reset_option('display.max_rows',None)

# # pd.set_option('display.max_rows', None)  # Show all rows


In [18]:
print(returns.loc[(returns.index.year == 2019) & (returns.index.month == 1)]['SP500'].std())

monthly_std = returns.groupby(by=[returns.index.month, returns.index.year]).std()
monthly_std.index = pd.to_datetime(monthly_std.index.map(lambda x: f"{x[1]}-{x[0]}"), format="%Y-%m")
monthly_std.sort_index(inplace=True)

monthly_std.index = returns_mon.index

0.011677586389697103


In [26]:
result = pd.concat([returns_mon, monthly_std], axis=1, keys=['Returns', 'StdDev'])
result.columns = result.columns.get_level_values(1) + '_' + result.columns.get_level_values(0).astype(str)


result


,AAPL_Returns,AVGO_Returns,AXP_Returns,BAC_Returns,C_Returns,CRM_Returns,CSCO_Returns,DOCU_Returns,GS_Returns,HUBS_Returns,...,CSCO_StdDev,DOCU_StdDev,GS_StdDev,HUBS_StdDev,JPM_StdDev,MSFT_StdDev,NOW_StdDev,QCOM_StdDev,TXN_StdDev,SP500_StdDev
Date,,,,,,,,,,,,,,,,,,,,,
2018-04-30,0.018113,-0.014434,-0.008634,-0.007629,-0.010436,0.006405,-0.009394,-0.027687,-0.006130,-0.018081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-31,0.135124,0.098727,-0.004557,-0.025446,-0.018520,0.068931,-0.035674,0.289412,-0.048887,0.144476,...,0.012305,0.026296,0.010957,0.015443,0.014172,0.008415,0.013723,0.015838,0.010325,0.006550
2018-06-30,-0.009418,-0.024502,-0.003052,-0.029270,0.003448,0.054666,0.007492,0.063040,-0.023508,0.034654,...,0.010630,0.038368,0.009118,0.029476,0.010105,0.010327,0.018725,0.010730,0.012988,0.005575
2018-07-31,0.027983,-0.086012,0.019156,0.095424,0.074268,0.005499,-0.009537,0.017941,0.076438,-0.010367,...,0.013935,0.032875,0.009614,0.018243,0.013648,0.013346,0.022173,0.020328,0.012673,0.005448
2018-08-31,0.200422,-0.012355,0.064911,0.001619,-0.002795,0.113234,0.129582,0.158442,0.004918,0.157937,...,0.008982,0.031341,0.011369,0.021076,0.007395,0.007513,0.015087,0.009310,0.013441,0.004431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,0.032353,0.013319,0.022170,0.010915,-0.025235,-0.022798,0.043137,0.067231,0.008327,0.004104,...,0.020368,0.020370,0.018847,0.018766,0.014808,0.012256,0.015843,0.031815,0.025276,0.012262
2024-09-30,0.017467,0.062937,0.048521,-0.019909,-0.000639,0.083985,0.053027,0.048640,-0.029672,0.065181,...,0.009361,0.022923,0.019290,0.015217,0.015982,0.011563,0.017725,0.024127,0.021164,0.008719
2024-10-31,-0.030429,-0.015826,-0.001528,0.053931,0.025080,0.064521,0.036989,0.117410,0.045808,0.043623,...,0.011385,0.019964,0.012744,0.018603,0.014478,0.015659,0.017850,0.022151,0.018249,0.006936


In [27]:
rf = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']].div(100)

rf.index = rf.index.to_timestamp(how='end').normalize()
rf

/var/folders/hn/ds6pkb3n5j16s47f26mbf8x40000gn/T/ipykernel_15161/2736375131.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rf = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']].div(100)
/var/folders/hn/ds6pkb3n5j16s47f26mbf8x40000gn/T/ipykernel_15161/2736375131.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rf = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']].div(100)


,RF
Date,
2000-01-31,0.0041
2000-02-29,0.0043
2000-03-31,0.0047
2000-04-30,0.0046
2000-05-31,0.0050
...,...
2024-08-31,0.0048
2024-09-30,0.0040
2024-10-31,0.0039


In [28]:
result = pd.merge(result, rf, how='left', left_index=True, right_index=True)

In [29]:
# Fetch CPI data from FRED
cpi = web.DataReader('CPIAUCSL', 'fred', start, end)

# Calculate monthly inflation rates as percentage change
inflation_rate = cpi.pct_change().dropna()

# Optionally normalize the index to end-of-month timestamps
inflation_rate.index = inflation_rate.index.to_period('M').to_timestamp(how='end').normalize()

inflation_rate.columns = ['Inflation Rate']
inflation_rate

,Inflation Rate
DATE,
2000-02-29,0.004135
2000-03-31,0.005882
2000-04-30,-0.000585
2000-05-31,0.001755
2000-06-30,0.005841
...,...
2024-08-31,0.001802
2024-09-30,0.002292
2024-10-31,0.002265


In [30]:
result = pd.merge(result, inflation_rate, how = 'left', left_index=True, right_index=True) 

In [32]:
result

,AAPL_Returns,AVGO_Returns,AXP_Returns,BAC_Returns,C_Returns,CRM_Returns,CSCO_Returns,DOCU_Returns,GS_Returns,HUBS_Returns,...,GS_StdDev,HUBS_StdDev,JPM_StdDev,MSFT_StdDev,NOW_StdDev,QCOM_StdDev,TXN_StdDev,SP500_StdDev,RF,Inflation Rate
Date,,,,,,,,,,,,,,,,,,,,,
2018-04-30,0.018113,-0.014434,-0.008634,-0.007629,-0.010436,0.006405,-0.009394,-0.027687,-0.006130,-0.018081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0014,0.002604
2018-05-31,0.135124,0.098727,-0.004557,-0.025446,-0.018520,0.068931,-0.035674,0.289412,-0.048887,0.144476,...,0.010957,0.015443,0.014172,0.008415,0.013723,0.015838,0.010325,0.006550,0.0014,0.002258
2018-06-30,-0.009418,-0.024502,-0.003052,-0.029270,0.003448,0.054666,0.007492,0.063040,-0.023508,0.034654,...,0.009118,0.029476,0.010105,0.010327,0.018725,0.010730,0.012988,0.005575,0.0014,0.000901
2018-07-31,0.027983,-0.086012,0.019156,0.095424,0.074268,0.005499,-0.009537,0.017941,0.076438,-0.010367,...,0.009614,0.018243,0.013648,0.013346,0.022173,0.020328,0.012673,0.005448,0.0016,0.000781
2018-08-31,0.200422,-0.012355,0.064911,0.001619,-0.002795,0.113234,0.129582,0.158442,0.004918,0.157937,...,0.011369,0.021076,0.007395,0.007513,0.015087,0.009310,0.013441,0.004431,0.0016,0.001787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,0.032353,0.013319,0.022170,0.010915,-0.025235,-0.022798,0.043137,0.067231,0.008327,0.004104,...,0.018847,0.018766,0.014808,0.012256,0.015843,0.031815,0.025276,0.012262,0.0048,0.001802
2024-09-30,0.017467,0.062937,0.048521,-0.019909,-0.000639,0.083985,0.053027,0.048640,-0.029672,0.065181,...,0.019290,0.015217,0.015982,0.011563,0.017725,0.024127,0.021164,0.008719,0.0040,0.002292
2024-10-31,-0.030429,-0.015826,-0.001528,0.053931,0.025080,0.064521,0.036989,0.117410,0.045808,0.043623,...,0.012744,0.018603,0.014478,0.015659,0.017850,0.022151,0.018249,0.006936,0.0039,0.002265


In [46]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

In [47]:
rolling_avg = returns.rolling(window=30).mean()
rolling_avg

,AAPL,AVGO,AXP,BAC,C,CRM,CSCO,DOCU,GS,HUBS,JPM,MSFT,NOW,QCOM,TXN,SP500
Date,,,,,,,,,,,,,,,,
2018-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,0.004749,0.011294,0.001273,-0.001009,0.000635,0.000630,0.000727,0.006606,-0.000845,0.002161,0.000479,0.001798,0.002351,-0.001431,-0.003712,0.000244
2024-12-26,0.004855,0.012585,0.001838,-0.000723,0.001171,0.000424,0.000754,0.006416,-0.000395,0.001308,0.000556,0.001305,0.001761,-0.000638,-0.003358,0.000327
2024-12-27,0.004281,0.012594,0.001616,-0.000887,0.001071,0.000050,0.000282,0.005719,-0.000776,0.000604,0.000063,0.000557,0.001313,-0.000299,-0.002347,-0.000049


In [52]:
result = pd.merge(result, rolling_avg, left_index=True, right_index=True, suffixes=("", "_RollingAvg"))



In [53]:
result

,AAPL_Returns,AVGO_Returns,AXP_Returns,BAC_Returns,C_Returns,CRM_Returns,CSCO_Returns,DOCU_Returns,GS_Returns,HUBS_Returns,...,CSCO_RollingAvg,DOCU_RollingAvg,GS_RollingAvg,HUBS_RollingAvg,JPM_RollingAvg,MSFT_RollingAvg,NOW_RollingAvg,QCOM_RollingAvg,TXN_RollingAvg,SP500_RollingAvg
Date,,,,,,,,,,,,,,,,,,,,,
2018-04-30,0.018113,-0.014434,-0.008634,-0.007629,-0.010436,0.006405,-0.009394,-0.027687,-0.006130,-0.018081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-31,0.135124,0.098727,-0.004557,-0.025446,-0.018520,0.068931,-0.035674,0.289412,-0.048887,0.144476,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-07-31,0.027983,-0.086012,0.019156,0.095424,0.074268,0.005499,-0.009537,0.017941,0.076438,-0.010367,...,-0.001139,-0.005023,0.000911,-0.003804,0.002290,0.001763,-0.001706,0.002819,-0.000642,0.000524
2018-08-31,0.200422,-0.012355,0.064911,0.001619,-0.002795,0.113234,0.129582,0.158442,0.004918,0.157937,...,0.004341,0.006517,0.001031,0.002738,0.001008,0.002034,0.001099,0.005438,-0.000507,0.001177
2018-10-31,-0.030477,-0.094192,-0.031827,-0.066531,-0.087539,-0.137018,-0.053252,-0.202207,0.005039,-0.101358,...,-0.000708,-0.006161,-0.001325,-0.002574,-0.002205,-0.001232,-0.000399,-0.005280,-0.004320,-0.002250
2018-11-30,-0.181198,0.062285,0.092865,0.032727,-0.003423,0.040222,0.046339,-0.004292,-0.150454,0.024917,...,0.001936,-0.002790,-0.005094,0.002039,0.001038,0.001147,0.000716,-0.003463,0.000702,-0.000022
2018-12-31,-0.116699,0.082345,-0.150975,-0.127546,-0.196481,-0.040558,-0.094840,-0.040230,-0.123971,-0.095663,...,-0.000477,0.000569,-0.006046,0.000792,-0.003012,-0.000782,0.002670,0.002280,0.000234,-0.002354
2019-01-31,0.055154,0.054939,0.081833,0.155438,0.238187,0.109513,0.099842,0.233782,0.185334,0.259127,...,0.002689,0.006090,0.005798,0.008254,0.001847,0.000761,0.008987,-0.004513,0.003649,0.002129
2019-02-28,0.044776,0.026505,0.049075,0.026641,-0.000469,0.076857,0.094734,0.115268,-0.002604,0.063609,...,0.005459,0.007008,0.003320,0.004945,0.000911,0.002394,0.008410,-0.001333,0.003105,0.002177
